# Tema 3: Recuperación de Información

## Importaciones

In [1]:
import numpy as np
import pandas as pd
import spacy
from gensim.models import KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from pathlib import Path


nlp = spacy.load("es_core_news_sm")
pd.set_option('max_colwidth', 400)

# Configurar rutas
PATH_DATA = Path.cwd().parent / 'data'
PATH_MODELS = Path.cwd().parent / 'models'


## Ejercicio 1

### Conjunto de Datos

Obtenido de https://github.com/jpposadas/FakeNewsCorpusSpanish

In [2]:
df = pd.read_excel(PATH_DATA / "train.xlsx", engine="openpyxl")

### Preprocesamiento

#### Minúsculas

In [3]:
headlines = []
for text in tqdm(df['Headline'].values):
    sent_in_tokens = []
    for token in nlp(text):
        sent_in_tokens.append(token.text.lower())
    headlines.append(sent_in_tokens)

texts = []
for text in tqdm(df['Text'].values):
    sent_in_tokens = []
    for token in nlp(text):
            sent_in_tokens.append(token.text.lower())
    texts.append(sent_in_tokens)

100%|██████████| 676/676 [00:20<00:00, 33.47it/s]


#### Filtrar sustantivos + verbos + adjetivos

In [4]:
headlines_filtered = []
for text in tqdm(df['Headline'].values):
    sent_in_tokens = []
    for token in nlp(text):
        if token.pos_ in ['NOUN', 'ADJ', 'VERB']:
            sent_in_tokens.append(token.text.lower())
    headlines_filtered.append(sent_in_tokens)

texts_filtered = []
for text in tqdm(df['Text'].values):
    sent_in_tokens = []
    for token in nlp(text):
        if token.pos_ in ['NOUN', 'ADJ', 'VERB']:
            sent_in_tokens.append(token.text.lower())
    texts_filtered.append(sent_in_tokens)

100%|██████████| 676/676 [00:19<00:00, 35.57it/s]


#### Filtrar sustantivos + verbos + adjetivos y lematizar

In [5]:
headlines_filtered_lemmas = []
for text in tqdm(df['Headline'].values):
    sent_in_tokens = []
    for token in nlp(text):
        if token.pos_ in ['NOUN', 'ADJ', 'VERB']:
            sent_in_tokens.append(token.lemma_.lower())
    headlines_filtered_lemmas.append(sent_in_tokens)

texts_filtered_lemmas = []
for text in tqdm(df['Text'].values):
    sent_in_tokens = []
    for token in nlp(text):
        if token.pos_ in ['NOUN', 'ADJ', 'VERB']:
            sent_in_tokens.append(token.lemma_.lower())
    texts_filtered_lemmas.append(sent_in_tokens)

100%|██████████| 676/676 [00:18<00:00, 35.90it/s]


### Recuperación de información

In [6]:
def headlines_results_dataframe(most_similar_indices, sim_row):
    return pd.DataFrame({
        'Headline': df['Headline'].iloc[most_similar_indices].values,
        'Similarity': sim_row[most_similar_indices],
        'Category': df['Category'].iloc[most_similar_indices].values,
        'Topic': df['Topic'].iloc[most_similar_indices].values
    })


def text_results_dataframe(most_similar_indices, sim_row):
    return pd.DataFrame({
        'Text': df['Text'].iloc[most_similar_indices].values,
        'Similarity': sim_row[most_similar_indices],
        'Category': df['Category'].iloc[most_similar_indices].values,
        'Topic': df['Topic'].iloc[most_similar_indices].values
    })

In [7]:
print('Query: ', headlines_filtered[1])

print()
print('filtering')
for i, title in enumerate(headlines_filtered):
    if 'palabra' in title:
        print(i, title)

print()
print('filtering + lemmas')
for i, title in enumerate(headlines_filtered_lemmas):
    if 'palabra' in title:
        print(i, title)

Query:  ['palabra', 'aceptada']

filtering
0 ['palabra', 'diccionario', 'definición', 'mujer', 'problemática']
1 ['palabra', 'aceptada']
73 ['registra', 'palabra', 'taco', 'tacos', 'mexicanos', 'cambiar', 'nombre']

filtering + lemmas
0 ['palabra', 'diccionario', 'definición', 'mujer', 'problemático']
1 ['palabra', 'aceptado']
8 ['*', 'palabra', 'creíar', 'inaceptable', 'aprobar']
73 ['registrar', 'palabra', 'taco', 'taco', 'mexicano', 'cambiar', 'nombre']


### TF-IDF

#### Minúsculas

In [8]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf.fit_transform(headlines)
sim_tfidf = cosine_similarity(X_tfidf, X_tfidf)

doc_id = 1
sim_row = sim_tfidf[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 2 ok

,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""",0.399241,Fake,Education
2,"""Está en un manicomio"" y ""una pesadilla"", lo que vive Lionel Messi en la selección argentina, según la TV española",0.243476,True,Sport
3,"""EL CHICHARITO"" SUSTITUIRÁ A CRISTIANO RONALDO COMO LA IMAGEN DE LA ROPA INTERIOR ""CR7 UNDERWEAR""",0.229093,Fake,Entertainment
4,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre",0.218512,Fake,Society


In [9]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf.fit_transform(texts)
sim_tfidf = cosine_similarity(X_tfidf, X_tfidf)

doc_id = 1
sim_row = sim_tfidf[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 1 ok

,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.383949,True,Education
2,"La libertad de expresión no mata periodistas\nMADRID (apro).— El premio Nobel de Literatura, Mario Vargas Llosa, no engaña cuando opina sobre el acontecer internacional, como un portavoz de mucha influencia de los círculos más conservadores del establishment español, púlpito desde el cual lanza su permanente cruzada contra del régimen venezolano, contra el populismo y los nacionalismos.\nEs vá...",0.310701,True,Politics
3,"Carlos Salinas y el TLCAN: México ya cedió la banca y el petróleo ""sin obtener nada en reciprocidad""\nEn las negociaciones para modernizar el Tratado de Libre Comercio de América del Norte (TLCAN), Estados Unidos está obligado a ceder en materia migratoria, pues México ya abrió a la inversión extranjera dos sectores estratégicos, la banca y el petrolero, sin haber obtenido ""nada en reciprocida...",0.307675,True,Politics
4,"Angélica Rivera es, en sí misma, un conflicto de interés de Peña Nieto\nAl margen de la increíble historia del financiamiento del Grupo Higa para la adquisición de la residencia de Sierra Gorda por parte de Angélica Rivera, el video en el cual ésta explica cómo adquirió sus inmuebles tiene un dato, uno más, que debe destacarse: Afirma que en *NUMBER* Televisa le entregó para su ""uso y gozo"" la...",0.304417,True,Politics


#### Filtrar sustantivos + verbos + adjetivos

In [10]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf.fit_transform(headlines_filtered)
sim_tfidf = cosine_similarity(X_tfidf, X_tfidf)

doc_id = 1
sim_row = sim_tfidf[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 2 ok

,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""",0.285763,Fake,Education
2,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre",0.248765,Fake,Society
3,Emitió BdeM billete conmemorativo del centenario de la Constitución,0.000000,True,Economy
4,Banxico pronostica crecimiento económico si López Obrador gana las elecciones este primero de Julio,0.000000,Fake,Economy


In [11]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf.fit_transform(texts_filtered)
sim_tfidf = cosine_similarity(X_tfidf, X_tfidf)

doc_id = 1
sim_row = sim_tfidf[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 3 ok

,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.073648,True,Education
2,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""\nEspaña.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término ""Lady"" como una nueva palabra en el diccionario del idioma español.\nDarío señaló que ""Lady"" servirá para definir a una ""muje...",0.072305,Fake,Education
3,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre\nOtra tremenda noticia empantana aún más las frágiles relaciones entre México y los Estados Unidos. Esta vez se trata de algo con pocos precedentes, pues la cadena de comida rápida Taco Bell, logró el registro a nivel mundial de la palabra ""taco"". Con esto, las taquerías de México tendrán que ponerle otro nomb...",0.057713,Fake,Society
4,"Sólo hijos de Ricardo Anaya estudian en el extranjero\nLíderes de otros partidos rechazan que sus familias vivan fuera del país; los de AMLO y Dante Delgado, con estudios universitarios, pero en México\nEl presidente nacional del PAN, Ricardo Anaya Cortés, con su familia en Atlanta, Georgia, Estados Unidos Los líderes de las principales fuerzas políticas del país manifestaron que no mantienen ...",0.057461,True,Entertainment


#### Filtrar sustantivos + verbos + adjetivos y lematizar

In [12]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf.fit_transform(headlines_filtered_lemmas)
sim_tfidf = cosine_similarity(X_tfidf, X_tfidf)

doc_id = 1
sim_row = sim_tfidf[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 3 ok

,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba,0.298954,True,Education
2,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""",0.276874,Fake,Education
3,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre",0.216931,Fake,Society
4,Emitió BdeM billete conmemorativo del centenario de la Constitución,0.000000,True,Economy


In [13]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf.fit_transform(texts_filtered_lemmas)
sim_tfidf = cosine_similarity(X_tfidf, X_tfidf)

doc_id = 1
sim_row = sim_tfidf[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 3 ok

,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.132668,True,Education
2,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""\nEspaña.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término ""Lady"" como una nueva palabra en el diccionario del idioma español.\nDarío señaló que ""Lady"" servirá para definir a una ""muje...",0.090664,Fake,Education
3,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre\nOtra tremenda noticia empantana aún más las frágiles relaciones entre México y los Estados Unidos. Esta vez se trata de algo con pocos precedentes, pues la cadena de comida rápida Taco Bell, logró el registro a nivel mundial de la palabra ""taco"". Con esto, las taquerías de México tendrán que ponerle otro nomb...",0.075764,Fake,Society
4,"""Desamigar"" en Facebook se considerará como discriminación\nEl Consejo Nacional para Prevenir la Discriminación (CONAPRED) anunció que, a partir de este mes de diciembre, el retirar a una persona de tu lista de amigos de Facebook puede considerarse como un acto de discriminación, pues según esta institución, atenta contra el derecho de las personas de expresarse libremente. Según un representa...",0.075674,Fake,Society


## Ejercicio 2

### Apartado 2.1: Word2Vec


In [14]:
def get_embedding_features(model, docs):
    features = []
    for doc in tqdm(docs):
        embeddings = [model[token] for token in doc if token in model]
        if embeddings:
            features.append(np.mean(embeddings, axis=0))
        else:
            features.append(np.zeros(model.vector_size))
    return np.array(features)

In [15]:
model = KeyedVectors.load_word2vec_format(str(PATH_MODELS / 'SBW-vectors-300-min5.txt'), binary=False)

#### Minúsculas

In [16]:
X_w2v = get_embedding_features(model, headlines)
sim_w2v = cosine_similarity(X_w2v, X_w2v)

doc_id = 1
sim_row = sim_w2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 1 ok

100%|██████████| 676/676 [00:00<00:00, 30162.65it/s]


,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""",0.868844,Fake,Education
2,La Casa Blanca pone por error la bandera mexicana en la recepción a Rajoy,0.860659,Fake,Politics
3,"Selena Gómez se convierte en la primera ""mexicana"" deportada de EU por Trump",0.852657,Fake,Entertainment
4,LA MARINA EXIGE LA RENUNCIA DE EPN POR SU INEPTITUD.,0.850623,Fake,Politics


In [17]:
X_w2v = get_embedding_features(model, texts)
sim_w2v = cosine_similarity(X_w2v, X_w2v)

doc_id = 1
sim_row = sim_w2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 2 ok

100%|██████████| 676/676 [00:00<00:00, 2266.41it/s]


,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.990061,True,Education
2,"EPN lanza nuevo billete y no creerás quien es la imagen\nMéxico.- En conferencia de prensa el gobernador del Banco de México, Agustín Carstens, presentó a los medios un nuevo billete mexicano ""edición especial"" que tiene una denominación de USD*NUMBER* pesos, con la finalidad de -según sus palabras- ""hacer más eficiente la compra de productos e incentivar el consumo local"", ya que con esta mon...",0.984612,Fake,Economy
3,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""\nEspaña.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término ""Lady"" como una nueva palabra en el diccionario del idioma español.\nDarío señaló que ""Lady"" servirá para definir a una ""muje...",0.984554,Fake,Education
4,"La libertad de expresión no mata periodistas\nMADRID (apro).— El premio Nobel de Literatura, Mario Vargas Llosa, no engaña cuando opina sobre el acontecer internacional, como un portavoz de mucha influencia de los círculos más conservadores del establishment español, púlpito desde el cual lanza su permanente cruzada contra del régimen venezolano, contra el populismo y los nacionalismos.\nEs vá...",0.984316,True,Politics


#### Filtrar sustantivos + verbos + adjetivos

In [18]:
X_w2v = get_embedding_features(model, headlines_filtered)
sim_w2v = cosine_similarity(X_w2v, X_w2v)

doc_id = 1
sim_row = sim_w2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 4 ok

100%|██████████| 676/676 [00:00<00:00, 52395.86it/s]


,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""",0.714154,Fake,Education
2,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre",0.625822,Fake,Society
3,"Aunque ya fue aprobado, los americanistas rechazan el cambio de nombre para el Estadio Azteca",0.604719,Fake,Society
4,Tenemos la lista completa de expresiones prohibidas en los estadios enviada por la FIFA a México,0.532307,Fake,Sport


In [19]:
X_w2v = get_embedding_features(model, texts_filtered)
sim_w2v = cosine_similarity(X_w2v, X_w2v)

doc_id = 1
sim_row = sim_w2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 2 ok

100%|██████████| 676/676 [00:00<00:00, 4243.38it/s]


,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.966267,True,Education
2,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""\nEspaña.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término ""Lady"" como una nueva palabra en el diccionario del idioma español.\nDarío señaló que ""Lady"" servirá para definir a una ""muje...",0.940075,Fake,Education
3,Un audio robado hunde a Pablo Iglesias: cócteles Molotov contra la Guardia Civil\nSe está haciendo viral tras recuperarse una grabación del líder podemita que su formación ha tratado de eliminar todo rastro. Machismo puro y violencia quedan retratados en este documento.\nSin duda se está haciendo viral. Es una grabación que corresponde a la presencia de Pablo Iglesias en la Escuela de Verano d...,0.934777,Fake,Society
4,"WhatsApp-Facebook: la unión hace la fuerza a costa del usuario\nLa popular aplicación de mensajería instantánea cambia por primera vez en cuatro años sus términos de uso de la plataforma\nPocas empresas son capaces de invertir (o gastar) *NUMBER* millones de euros en una aplicación para dispositivos móviles. Pocas, por no decir ninguna, verían negocio en tal operación financiera. Facebook sí l...",0.927836,True,Science


#### Filtrar sustantivos + verbos + adjetivos y lematizar

In [20]:
X_w2v = get_embedding_features(model, headlines_filtered_lemmas)
sim_w2v = cosine_similarity(X_w2v, X_w2v)

doc_id = 1
sim_row = sim_w2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 4 ok

100%|██████████| 676/676 [00:00<00:00, 25665.78it/s]


,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""",0.727326,Fake,Education
2,*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba,0.726046,True,Education
3,"Taco Bell registra la palabra ""taco"" — Los tacos mexicanos deberán cambiar de nombre",0.622879,Fake,Society
4,"Aunque ya fue aprobado, los americanistas rechazan el cambio de nombre para el Estadio Azteca",0.618925,Fake,Society


In [21]:
X_w2v = get_embedding_features(model, texts_filtered_lemmas)
sim_w2v = cosine_similarity(X_w2v, X_w2v)

doc_id = 1
sim_row = sim_w2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 2 ok

100%|██████████| 676/676 [00:00<00:00, 3451.01it/s]


,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.973292,True,Education
2,"WhatsApp-Facebook: la unión hace la fuerza a costa del usuario\nLa popular aplicación de mensajería instantánea cambia por primera vez en cuatro años sus términos de uso de la plataforma\nPocas empresas son capaces de invertir (o gastar) *NUMBER* millones de euros en una aplicación para dispositivos móviles. Pocas, por no decir ninguna, verían negocio en tal operación financiera. Facebook sí l...",0.949703,True,Science
3,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONARIO DEL IDIOMA ESPAÑOL COMO DEFINICIÓN DE ""MUJER PROBLEMÁTICA""\nEspaña.- El presidente de la Real Academia Española (RAE), Darío Villanueva, informó en conferencia de prensa que a partir del próximo mes se incluirá el término ""Lady"" como una nueva palabra en el diccionario del idioma español.\nDarío señaló que ""Lady"" servirá para definir a una ""muje...",0.947912,Fake,Education
4,Un audio robado hunde a Pablo Iglesias: cócteles Molotov contra la Guardia Civil\nSe está haciendo viral tras recuperarse una grabación del líder podemita que su formación ha tratado de eliminar todo rastro. Machismo puro y violencia quedan retratados en este documento.\nSin duda se está haciendo viral. Es una grabación que corresponde a la presencia de Pablo Iglesias en la Escuela de Verano d...,0.943012,Fake,Society


### Apartado 2.2: Doc2Vec


#### Minúsculas

In [22]:
documents = []
for i, headline in enumerate(headlines):
    documents.append(TaggedDocument(words=headline, tags=[i]))

model = Doc2Vec(vector_size=50, epochs=40, seed=42)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

X_d2v = [model.infer_vector(headline) for headline in headlines]
sim_d2v = cosine_similarity(X_d2v, X_d2v)

doc_id = 1
sim_row = sim_d2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 0 ok

,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,"Escandaloso audio de Andrea Legarreta ""Me dan asco los pobretones""",0.992345,Fake,Entertainment
2,"Inadmisible e irresponsable, plagio en tesis de EPN: Krauze",0.990106,True,Politics
3,Tu perro también liga con el móvil,0.989985,True,Science
4,DURO GOLPE: Despiden a Denise Maerker por conflictiva e ineficiente,0.989808,Fake,Entertainment


In [23]:
documents = []
for i, text in enumerate(texts):
    documents.append(TaggedDocument(words=text, tags=[i]))

model = Doc2Vec(vector_size=50, epochs=40, seed=42)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

X_d2v = [model.infer_vector(text) for text in texts]
sim_d2v = cosine_similarity(X_d2v, X_d2v)

doc_id = 1
sim_row = sim_d2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 1 ok

,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.640494,True,Education
2,"Las frutas que contagiaron VIH en *NUMBER*\nHay mensajes de alerta circulando por todas las redes sociales donde te advierten que tengas cuidado con las naranjas y plátanos, ya que muchas de estas frutas han sido inyectadas con sangre portadora del VIH (virus de inmunodeficiencia humana) que una vez activo desarrolla el SIDA. La noticia se ha compartido más de *NUMBER* veces desde el *NUMBER* ...",0.515225,Fake,Health
3,"COREA DEL NORTE EJECUTA A UN TRAIDOR MEDIANTE ""MUERTE POR KIKI""\nEl gobierno de Corea del Norte sigue practicando ejecuciones a traidores, emplean varios métodos de ejecución, variados para satisfacer las curiosidades de Kim Jong-un, al que le gusta innovar en todas las materias. El gobierno surcoreano sigue ejecutando simpatizantes del tío de Kim Jong-un, y lo nuevo es ejecutarlos mediante mu...",0.501157,Fake,Politics
4,"La Conapred defiende a Darwin Quintero y pide que no salga del América\nDespués de que el pasado fin de semana los Tigres le pasaron por encima al Club América en la ronda de semifinales de la Liga MX, dejando a los de Coapa en el camino por obtener un título más para sus vitrinas, se ha estado en busca de un chivo expiatorio.\nY no sólo porque no llegar a la final es un fracaso, sino porque e...",0.493747,Fake,Sport


#### Filtrar sustantivos + verbos + adjetivos

In [24]:
documents = []
for i, headline in enumerate(headlines_filtered):
    documents.append(TaggedDocument(words=headline, tags=[i]))

model = Doc2Vec(vector_size=50, epochs=40, seed=42)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

X_d2v = [model.infer_vector(headline) for headline in headlines_filtered]
sim_d2v = cosine_similarity(X_d2v, X_d2v)

doc_id = 1
sim_row = sim_d2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 1 ok

,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,La autoridad electoral mexicana dará resultados a cuentagotas el *NUMBER* de julio,0.493956,True,Politics
2,MUERE MANNY PACQUIAO EN FILIPINAS,0.349747,Fake,Sport
3,Madrid abre el primer Centro para el tratamiento de la Homosexualidad,0.341884,Fake,Society
4,Canadá denuncia a Estados Unidos ante la OMC,0.339360,True,Politics


In [25]:
documents = []
for i, text in enumerate(texts_filtered):
    documents.append(TaggedDocument(words=text, tags=[i]))

model = Doc2Vec(vector_size=50, epochs=40, seed=42)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

X_d2v = [model.infer_vector(text) for text in texts_filtered]
sim_d2v = cosine_similarity(X_d2v, X_d2v)

doc_id = 1
sim_row = sim_d2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 1 ok

,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.599806,True,Education
2,"Messi: ""Todavía estoy a tiempo de jugar con la selección española""\n En una entrevista aparecida hoy en el diario Sport de Barcelona, el astro argentino se sincera y asegura que se nacionalizaría español si le permitieran jugar en la selección española.\nUno de los retos que le quedan al mejor jugador de la historia del fútbol moderno, Leo Messi, es ganar una copa del mundo de fútbol. Sin emba...",0.550553,Fake,Sport
3,"El Partido Popular denuncia a la marca ""Chorizos PP"" por desprestigiar su imagen\nEl Partido Popular ha interpuesto una demanda contra la marca ""Chorizos PP"" de León porque según ellos ""Es una clara mofa hacia nuestro nombre y lu único que pretende es desprestigiar la marca de nuestro partido"" según se explica en la denuncia.\nEl dueño de la marca se ha defendido de tales acusaciones alegando ...",0.526234,Fake,Politics
4,"TV Azteca anuncia que cambiará su nombre por uno más nacionalista\nComo una manera de asumir más la identidad nacional frente al embate del gobierno de Donald Trump, Grupo Salinas anunció que modificará los nombres de algunas de sus empresas.\nPara empezar, TV Azteca ahora se llamará TV Nahua, pues, argumentan, el nombre de ""azteca"" se refería a la mítica ciudad de Aztlán, la cual estaría ubic...",0.508011,Fake,Entertainment


#### Filtrar sustantivos + verbos + adjetivos y lematizar

In [26]:
documents = []
for i, headline in enumerate(headlines_filtered_lemmas):
    documents.append(TaggedDocument(words=headline, tags=[i]))

model = Doc2Vec(vector_size=50, epochs=40, seed=42)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

X_d2v = [model.infer_vector(headline) for headline in headlines_filtered_lemmas]
sim_d2v = cosine_similarity(X_d2v, X_d2v)

doc_id = 1
sim_row = sim_d2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
headlines_results_dataframe(most_similar_indices, sim_row) # 0 ok

,Headline,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE",1.000000,Fake,Education
1,Apoyo de Venezuela a AMLO,0.403254,Fake,Politics
2,"""Propone AMLO a Belinda para la Secretaría de Cultura"" ¡YA LE TIRÓ SU HUESO!",0.384244,Fake,Politics
3,Mexivergas es internado en un centro psiquiátrico,0.370029,Fake,Entertainment
4,"Alvaro Uribe ""Somos el poder y podemos reprimir cuando queramos""",0.335849,Fake,Politics


In [27]:
documents = []
for i, text in enumerate(texts_filtered_lemmas):
    documents.append(TaggedDocument(words=text, tags=[i]))

model = Doc2Vec(vector_size=50, epochs=40, seed=42)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

X_d2v = [model.infer_vector(text) for text in texts_filtered_lemmas]
sim_d2v = cosine_similarity(X_d2v, X_d2v)

doc_id = 1
sim_row = sim_d2v[doc_id]
most_similar_indices = list(sim_row.argsort()[-5:][::-1])
text_results_dataframe(most_similar_indices, sim_row) # 2 ok

,Text,Similarity,Category,Topic
0,"La palabra ""haiga"", aceptada por la RAE La Real Academia de la Lengua (RAE), ha aceptado el uso de ""HAIGA"", para su utilización en las tres personas del singular del presente del subjuntivo del verbo hacer, aunque asegura que la forma más recomendable en la lengua culta para este tiempo, sigue siendo ""haya"".\nAsí lo han confirmado fuentes de la RAE, que explican que este cambio ha sido propues...",1.000000,Fake,Education
1,"*NUMBER* palabras que creíamos inaceptables y que la RAE (sorprendentemente) aprueba\nLa cuenta de Twitter de la Real Academia Española hierve de actividad. Sus respuestas a las consultas de los usuarios pocas veces dejan indiferente\n""Consulta: cuál es la forma correcta de referirse a internet: la internet o el internet"". Esta es una de las cientos de preguntas que los lectores lanzan a la RA...",0.661649,True,Education
2,"El Gobierno ya ha elegido a los *NUMBER* españoles de derechas que cederán sus trabajos a los migrantes del Aquarius\nEL NUEVO LÍDER DEL PP CATALÁN SERÁ EL SUBSAHARIANO AKANNI NSONGAN\nLa vicepresidenta del Gobierno valenciano, Mónica Oltra, ha ofrecido esta mañana más detalles sobre la operación Esperanza Mediterráneo, con la que el Gobierno español acogerá a los migrantes a bordo del barco A...",0.575401,Fake,Politics
3,"Conapred investiga acto de racismo en el Pumas vs América\n""Simio"", es la palabra con la que Darío Verón, capitán de Pumas, se habría referido al americanista, Darwin Quintero.\nEl Consejo Nacional para Prevenir la Discriminación (Conapred) abrió una queja de oficio para investigar el supuesto acto racista en el juego de semifinal, entre Pumas y América, por parte del jugador Dario Verón.\nEl ...",0.525845,True,Sport
4,"Españoles o inmigrantes: las cifras reales de quién recibe más ayudas al alquiler\nLos datos desmienten la creencia xenófoba de que los extranjeros copan las subvenciones\nLa creencia xenófoba de que los extranjeros reciben más subvenciones que los españoles vuelve a cobrar fuerza con el anuncio del nuevo Plan de Vivienda, que contempla ayudas directas al alquiler para quienes paguen una renta...",0.475535,True,Politics
